In [6]:
import pygame
import heapq  

# Configuraciones iniciales PROGRAMA 5
ANCHO_VENTANA = 600
VENTANA = pygame.display.set_mode((ANCHO_VENTANA, ANCHO_VENTANA))
pygame.display.set_caption("Visualización de Nodos")

# Colores (RGB)
BLANCO = (255, 255, 255)
NEGRO = (0, 0, 0)
GRIS = (128, 128, 128)
VERDE = (0, 255, 0)
ROJO = (255, 0, 0)
NARANJA = (255, 165, 0)
PURPURA = (128, 0, 128)
AZUL = (0, 0, 255)  

class Nodo:
    def __init__(self, fila, col, ancho, total_filas):
        self.fila = fila
        self.col = col
        self.x = fila * ancho
        self.y = col * ancho
        self.color = BLANCO
        self.ancho = ancho
        self.total_filas = total_filas
        self.vecinos = []

    def get_pos(self):
        return self.fila, self.col

    def es_pared(self):
        return self.color == NEGRO

    def es_inicio(self):
        return self.color == NARANJA

    def es_fin(self):
        return self.color == PURPURA

    def restablecer(self):
        self.color = BLANCO

    def hacer_inicio(self):
        self.color = NARANJA

    def hacer_pared(self):
        self.color = NEGRO

    def hacer_fin(self):
        self.color = PURPURA

    def dibujar(self, ventana):
        pygame.draw.rect(ventana, self.color, (self.x, self.y, self.ancho, self.ancho))

    def actualizar_vecinos(self, grid):
        self.vecinos = []
        if self.fila < self.total_filas - 1 and not grid[self.fila + 1][self.col].es_pared():  # Abajo
            self.vecinos.append(grid[self.fila + 1][self.col])
        if self.fila > 0 and not grid[self.fila - 1][self.col].es_pared():  # Arriba
            self.vecinos.append(grid[self.fila - 1][self.col])
        if self.col < self.total_filas - 1 and not grid[self.fila][self.col + 1].es_pared():  # Derecha
            self.vecinos.append(grid[self.fila][self.col + 1])
        if self.col > 0 and not grid[self.fila][self.col - 1].es_pared():  # Izquierda
            self.vecinos.append(grid[self.fila][self.col - 1])

def crear_grid(filas, ancho):
    grid = []
    ancho_nodo = ancho // filas
    for i in range(filas):
        grid.append([])
        for j in range(filas):
            nodo = Nodo(i, j, ancho_nodo, filas)
            grid[i].append(nodo)
    return grid

def dibujar_grid(ventana, filas, ancho):
    ancho_nodo = ancho // filas
    for i in range(filas):
        pygame.draw.line(ventana, GRIS, (0, i * ancho_nodo), (ancho, i * ancho_nodo))
        for j in range(filas):
            pygame.draw.line(ventana, GRIS, (j * ancho_nodo, 0), (j * ancho_nodo, ancho))

def dibujar(ventana, grid, filas, ancho):
    ventana.fill(BLANCO)
    for fila in grid:
        for nodo in fila:
            nodo.dibujar(ventana)
    dibujar_grid(ventana, filas, ancho)
    pygame.display.update()

def obtener_click_pos(pos, filas, ancho):
    ancho_nodo = ancho // filas
    y, x = pos
    fila = y // ancho_nodo
    col = x // ancho_nodo
    return fila, col

def h(pos1, pos2):
    x1, y1 = pos1
    x2, y2 = pos2
    return abs(x1 - x2) + abs(y1 - y2)

def reconstruir_camino(came_from, current, dibujar):
    camino = []
    while current in came_from:
        camino.append(current)
        current = came_from[current]
        current.color = VERDE
        dibujar()
    return camino

def a_star(dibujar, grid, inicio, fin):
    count = 0
    open_set = []
    heapq.heappush(open_set, (0, count, inicio))
    came_from = {}
    g_score = {nodo: float("inf") for fila in grid for nodo in fila}
    g_score[inicio] = 0
    f_score = {nodo: float("inf") for fila in grid for nodo in fila}
    f_score[inicio] = h(inicio.get_pos(), fin.get_pos())

    open_set_hash = {inicio}
    closed_set = set()  # Lista cerrada para los nodos procesados
    lista_cerrada = []  # Para almacenar la lista cerrada

    while open_set:
        current = heapq.heappop(open_set)[2]
        open_set_hash.remove(current)

        if current == fin:
            camino = reconstruir_camino(came_from, fin, dibujar)
            # Imprimir lista cerrada y camino final
            print("Lista cerrada:", [f"[{nodo.fila}][{nodo.col}]" for nodo in lista_cerrada])
            print("R =", [f"[{nodo.fila}][{nodo.col}]" for nodo in camino])
            return True

        # Añadir a la lista cerrada y marcarlo como visitado
        closed_set.add(current)
        lista_cerrada.append(current)  # Añadir el nodo a la lista cerrada
        current.color = AZUL  # Marca el nodo como visitado
        dibujar()

        for vecino in current.vecinos:
            if vecino in closed_set:  # Si el vecino ya está en la lista cerrada, saltarlo
                continue

            temp_g_score = g_score[current] + 1

            if temp_g_score < g_score[vecino]:
                came_from[vecino] = current
                g_score[vecino] = temp_g_score
                f_score[vecino] = temp_g_score + h(vecino.get_pos(), fin.get_pos())

                if vecino not in open_set_hash:
                    count += 1
                    heapq.heappush(open_set, (f_score[vecino], count, vecino))
                    open_set_hash.add(vecino)

        dibujar()

    return False


def main(ventana, ancho):
    FILAS = 9
    grid = crear_grid(FILAS, ancho)
    inicio = None
    fin = None

    corriendo = True

    while corriendo:
        dibujar(ventana, grid, FILAS, ancho)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                corriendo = False

            if pygame.mouse.get_pressed()[0]:  # Click izquierdo
                pos = pygame.mouse.get_pos()
                fila, col = obtener_click_pos(pos, FILAS, ancho)
                nodo = grid[fila][col]
                if not inicio and nodo != fin:
                    inicio = nodo
                    inicio.hacer_inicio()

                elif not fin and nodo != inicio:
                    fin = nodo
                    fin.hacer_fin()

                elif nodo != fin and nodo != inicio:
                    nodo.hacer_pared()

            elif pygame.mouse.get_pressed()[2]:  # Click derecho
                pos = pygame.mouse.get_pos()
                fila, col = obtener_click_pos(pos, FILAS, ancho)
                nodo = grid[fila][col]
                nodo.restablecer()
                if nodo == inicio:
                    inicio = None
                elif nodo == fin:
                    fin = None

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and inicio and fin:
                    for fila in grid:
                        for nodo in fila:
                            nodo.actualizar_vecinos(grid)
                    a_star(lambda: dibujar(ventana, grid, FILAS, ancho), grid, inicio, fin)

    pygame.quit()

main(VENTANA, ANCHO_VENTANA)



In [ ]:
import pygame
import math
from queue import PriorityQueue

# Configuración de colores POSIBLE SOLUCION OPTIMA
ROJO = (255, 0, 0)
VERDE = (0, 255, 0)
AZUL = (0, 0, 255)
AMARILLO = (255, 255, 0)
BLANCO = (255, 255, 255)
NEGRO = (0, 0, 0)
GRIS = (128, 128, 128)
TURQUESA = (64, 224, 208)

# Tamaño de la ventana
ANCHO = 600
FILAS = 10

class Nodo:
    def __init__(self, fila, col, ancho, total_filas):
        self.fila = fila
        self.col = col
        self.x = fila * ancho
        self.y = col * ancho
        self.color = BLANCO
        self.vecinos = []
        self.ancho = ancho
        self.total_filas = total_filas

    def get_pos(self):
        return self.fila, self.col

    def es_pared(self):
        return self.color == NEGRO

    def reset(self):
        self.color = BLANCO

    def hacer_pared(self):
        self.color = NEGRO

    def dibujar(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.ancho, self.ancho))

    def actualizar_vecinos(self, grid):
        self.vecinos = []
        # Movimientos cardinales
        if self.fila < self.total_filas - 1 and not grid[self.fila + 1][self.col].es_pared():  # Abajo
            self.vecinos.append(grid[self.fila + 1][self.col])
        if self.fila > 0 and not grid[self.fila - 1][self.col].es_pared():  # Arriba
            self.vecinos.append(grid[self.fila - 1][self.col])
        if self.col < self.total_filas - 1 and not grid[self.fila][self.col + 1].es_pared():  # Derecha
            self.vecinos.append(grid[self.fila][self.col + 1])
        if self.col > 0 and not grid[self.fila][self.col - 1].es_pared():  # Izquierda
            self.vecinos.append(grid[self.fila][self.col - 1])

        # Movimientos diagonales
        if self.fila > 0 and self.col > 0 and not grid[self.fila - 1][self.col - 1].es_pared():  # Arriba-Izquierda
            self.vecinos.append(grid[self.fila - 1][self.col - 1])
        if self.fila > 0 and self.col < self.total_filas - 1 and not grid[self.fila - 1][self.col + 1].es_pared():  # Arriba-Derecha
            self.vecinos.append(grid[self.fila - 1][self.col + 1])
        if self.fila < self.total_filas - 1 and self.col > 0 and not grid[self.fila + 1][self.col - 1].es_pared():  # Abajo-Izquierda
            self.vecinos.append(grid[self.fila + 1][self.col - 1])
        if self.fila < self.total_filas - 1 and self.col < self.total_filas - 1 and not grid[self.fila + 1][self.col + 1].es_pared():  # Abajo-Derecha
            self.vecinos.append(grid[self.fila + 1][self.col + 1])

def h(p1, p2):
    # Heurística: distancia Manhattan
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)

def reconstruir_camino(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.color = AZUL
        draw()

def a_star(draw, grid, start, end):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {nodo: float("inf") for fila in grid for nodo in fila}
    g_score[start] = 0
    f_score = {nodo: float("inf") for fila in grid for nodo in fila}
    f_score[start] = h(start.get_pos(), end.get_pos())
    open_set_hash = {start}

    while not open_set.empty():
        current = open_set.get()[2]
        open_set_hash.remove(current)

        if current == end:
            reconstruir_camino(came_from, end, draw)
            return True

        for vecino in current.vecinos:
            es_diagonal = abs(vecino.fila - current.fila) + abs(vecino.col - current.col) == 2
            costo_movimiento = math.sqrt(2) if es_diagonal else 1
            temp_g_score = g_score[current] + costo_movimiento

            if temp_g_score < g_score[vecino]:
                came_from[vecino] = current
                g_score[vecino] = temp_g_score
                f_score[vecino] = temp_g_score + h(vecino.get_pos(), end.get_pos())

                if vecino not in open_set_hash:
                    count += 1
                    open_set.put((f_score[vecino], count, vecino))
                    open_set_hash.add(vecino)

        draw()
    return False

def crear_grid(filas, ancho):
    grid = []
    gap = ancho // filas
    for i in range(filas):
        grid.append([])
        for j in range(filas):
            nodo = Nodo(i, j, gap, filas)
            grid[i].append(nodo)
    return grid

def dibujar_grid(win, filas, ancho):
    gap = ancho // filas
    for i in range(filas):
        pygame.draw.line(win, GRIS, (0, i * gap), (ancho, i * gap))
        for j in range(filas):
            pygame.draw.line(win, GRIS, (j * gap, 0), (j * gap, ancho))

def dibujar(win, grid, filas, ancho):
    win.fill(BLANCO)
    for fila in grid:
        for nodo in fila:
            nodo.dibujar(win)
    dibujar_grid(win, filas, ancho)
    pygame.display.update()

def main(win, ancho):
    grid = crear_grid(FILAS, ancho)
    start, end = None, None

    run = True
    while run:
        dibujar(win, grid, FILAS, ancho)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if pygame.mouse.get_pressed()[0]:  # Izquierdo
                pos = pygame.mouse.get_pos()
                fila, col = pos[0] // (ancho // FILAS), pos[1] // (ancho // FILAS)
                nodo = grid[fila][col]
                if not start and nodo != end:
                    start = nodo
                    start.color = VERDE
                elif not end and nodo != start:
                    end = nodo
                    end.color = ROJO
                elif nodo != start and nodo != end:
                    nodo.hacer_pared()

            elif pygame.mouse.get_pressed()[2]:  # Derecho
                pos = pygame.mouse.get_pos()
                fila, col = pos[0] // (ancho // FILAS), pos[1] // (ancho // FILAS)
                nodo = grid[fila][col]
                nodo.reset()
                if nodo == start:
                    start = None
                elif nodo == end:
                    end = None

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start and end:
                    for fila in grid:
                        for nodo in fila:
                            nodo.actualizar_vecinos(grid)
                    a_star(lambda: dibujar(win, grid, FILAS, ancho), grid, start, end)

                if event.key == pygame.K_c:
                    start, end = None, None
                    grid = crear_grid(FILAS, ancho)

    pygame.quit()

win = pygame.display.set_mode((ANCHO, ANCHO))
pygame.display.set_caption("Algoritmo A* con movimientos diagonales")
main(win, ANCHO)


In [2]:
import pygame
import heapq  

# Configuraciones iniciales AHORA VAMOS A VERIFICAR ESTE CODIGO 
ANCHO_VENTANA = 600
VENTANA = pygame.display.set_mode((ANCHO_VENTANA, ANCHO_VENTANA))
pygame.display.set_caption("Visualización de Nodos")

# Colores (RGB)
BLANCO = (255, 255, 255)
NEGRO = (0, 0, 0)
GRIS = (128, 128, 128)
VERDE = (0, 255, 0)
ROJO = (255, 0, 0)
NARANJA = (255, 165, 0)
PURPURA = (128, 0, 128)
AZUL = (0, 0, 255)  

class Nodo:
    def __init__(self, fila, col, ancho, total_filas):
        self.fila = fila
        self.col = col
        self.x = fila * ancho
        self.y = col * ancho
        self.color = BLANCO
        self.ancho = ancho
        self.total_filas = total_filas
        self.vecinos = []

    def get_pos(self):
        return self.fila, self.col

    def es_pared(self):
        return self.color == NEGRO

    def es_inicio(self):
        return self.color == NARANJA

    def es_fin(self):
        return self.color == PURPURA

    def restablecer(self):
        self.color = BLANCO

    def hacer_inicio(self):
        self.color = NARANJA

    def hacer_pared(self):
        self.color = NEGRO

    def hacer_fin(self):
        self.color = PURPURA

    def dibujar(self, ventana):
        pygame.draw.rect(ventana, self.color, (self.x, self.y, self.ancho, self.ancho))

    def actualizar_vecinos(self, grid):
        self.vecinos = []
        if self.fila < self.total_filas - 1 and not grid[self.fila + 1][self.col].es_pared():  # Abajo
            self.vecinos.append(grid[self.fila + 1][self.col])
        if self.fila > 0 and not grid[self.fila - 1][self.col].es_pared():  # Arriba
            self.vecinos.append(grid[self.fila - 1][self.col])
        if self.col < self.total_filas - 1 and not grid[self.fila][self.col + 1].es_pared():  # Derecha
            self.vecinos.append(grid[self.fila][self.col + 1])
        if self.col > 0 and not grid[self.fila][self.col - 1].es_pared():  # Izquierda
            self.vecinos.append(grid[self.fila][self.col - 1])

def crear_grid(filas, ancho):
    grid = []
    ancho_nodo = ancho // filas
    for i in range(filas):
        grid.append([])
        for j in range(filas):
            nodo = Nodo(i, j, ancho_nodo, filas)
            grid[i].append(nodo)
    return grid

def dibujar_grid(ventana, filas, ancho):
    ancho_nodo = ancho // filas
    for i in range(filas):
        pygame.draw.line(ventana, GRIS, (0, i * ancho_nodo), (ancho, i * ancho_nodo))
        for j in range(filas):
            pygame.draw.line(ventana, GRIS, (j * ancho_nodo, 0), (j * ancho_nodo, ancho))

def dibujar(ventana, grid, filas, ancho):
    ventana.fill(BLANCO)
    for fila in grid:
        for nodo in fila:
            nodo.dibujar(ventana)
    dibujar_grid(ventana, filas, ancho)
    pygame.display.update()

def obtener_click_pos(pos, filas, ancho):
    ancho_nodo = ancho // filas
    y, x = pos
    fila = y // ancho_nodo
    col = x // ancho_nodo
    
    # Verificar que la fila y la columna estén dentro del rango de la grid
    fila = min(fila, filas - 1)
    col = min(col, filas - 1)

    return fila, col

def h(pos1, pos2):
    x1, y1 = pos1
    x2, y2 = pos2
    return abs(x1 - x2) + abs(y1 - y2)

def reconstruir_camino(came_from, current, dibujar):
    camino = []
    while current in came_from:
        camino.append(current)
        current = came_from[current]
        current.color = VERDE
        dibujar()
    return camino

def a_star(dibujar, grid, inicio, fin):
    count = 0
    open_set = []
    heapq.heappush(open_set, (0, count, inicio))
    came_from = {}
    g_score = {nodo: float("inf") for fila in grid for nodo in fila}
    g_score[inicio] = 0
    f_score = {nodo: float("inf") for fila in grid for nodo in fila}
    f_score[inicio] = h(inicio.get_pos(), fin.get_pos())

    open_set_hash = {inicio}
    closed_set = set()  # Lista cerrada para los nodos procesados
    lista_cerrada = []  # Para almacenar la lista cerrada

    while open_set:
        current = heapq.heappop(open_set)[2]
        open_set_hash.remove(current)

        if current == fin:
            camino = reconstruir_camino(came_from, fin, dibujar)
            # Imprimir lista cerrada y camino final
            print("Lista cerrada:", [f"[{nodo.fila}][{nodo.col}]" for nodo in lista_cerrada])
            print("R =", [f"[{nodo.fila}][{nodo.col}]" for nodo in camino])
            return True

        # Añadir a la lista cerrada y marcarlo como visitado
        closed_set.add(current)
        lista_cerrada.append(current)  # Añadir el nodo a la lista cerrada
        current.color = AZUL  # Marca el nodo como visitado
        dibujar()

        for vecino in current.vecinos:
            if vecino in closed_set:  # Si el vecino ya está en la lista cerrada, saltarlo
                continue

            temp_g_score = g_score[current] + 1

            if temp_g_score < g_score[vecino]:
                came_from[vecino] = current
                g_score[vecino] = temp_g_score
                f_score[vecino] = temp_g_score + h(vecino.get_pos(), fin.get_pos())

                if vecino not in open_set_hash:
                    count += 1
                    heapq.heappush(open_set, (f_score[vecino], count, vecino))
                    open_set_hash.add(vecino)

        dibujar()

    return False


def main(ventana, ancho):
    FILAS = 9
    grid = crear_grid(FILAS, ancho)
    inicio = None
    fin = None

    corriendo = True

    while corriendo:
        dibujar(ventana, grid, FILAS, ancho)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                corriendo = False

            if pygame.mouse.get_pressed()[0]:  # Click izquierdo
                pos = pygame.mouse.get_pos()
                fila, col = obtener_click_pos(pos, FILAS, ancho)
                nodo = grid[fila][col]
                if not inicio and nodo != fin:
                    inicio = nodo
                    inicio.hacer_inicio()

                elif not fin and nodo != inicio:
                    fin = nodo
                    fin.hacer_fin()

                elif nodo != fin and nodo != inicio:
                    nodo.hacer_pared()

            elif pygame.mouse.get_pressed()[2]:  # Click derecho
                pos = pygame.mouse.get_pos()
                fila, col = obtener_click_pos(pos, FILAS, ancho)
                nodo = grid[fila][col]
                nodo.restablecer()
                if nodo == inicio:
                    inicio = None
                elif nodo == fin:
                    fin = None

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and inicio and fin:
                    for fila in grid:
                        for nodo in fila:
                            nodo.actualizar_vecinos(grid)
                    a_star(lambda: dibujar(ventana, grid, FILAS, ancho), grid, inicio, fin)

    pygame.quit()

main(VENTANA, ANCHO_VENTANA)


Lista cerrada: ['[1][2]', '[2][2]', '[1][3]', '[3][2]', '[2][3]', '[1][4]', '[3][3]', '[2][4]', '[3][4]', '[0][2]', '[1][1]', '[2][1]', '[0][3]', '[3][1]', '[0][4]', '[1][5]', '[2][5]', '[3][5]', '[0][1]', '[1][0]', '[2][0]', '[3][0]', '[0][5]', '[1][6]', '[2][6]', '[3][6]', '[4][0]', '[5][0]', '[6][0]', '[5][1]', '[7][0]', '[6][1]', '[5][2]', '[8][0]', '[7][1]', '[6][2]', '[5][3]', '[8][1]', '[7][2]', '[6][3]', '[5][4]', '[8][2]', '[7][3]', '[6][4]', '[8][3]', '[7][4]']
R = ['[8][4]', '[8][3]', '[8][2]', '[8][1]', '[8][0]', '[7][0]', '[6][0]', '[5][0]', '[4][0]', '[3][0]', '[3][1]', '[3][2]', '[2][2]']


In [5]:
import pygame
from queue import PriorityQueue
import tkinter as tk
from threading import Thread
import string

# Configuraciones iniciales CODIGO EN VERIFICCIIJNOPI
ANCHO_VENTANA = 600

# Colores (RGB)
BLANCO = (255, 255, 255)
NEGRO = (0, 0, 0)
GRIS = (128, 128, 128)
VERDE = (0, 255, 0)
ROJO = (255, 0, 0)
NARANJA = (255, 165, 0)
PURPURA = (128, 0, 128)
AZUL = (0, 0, 255)

pygame.font.init()
FUENTE = pygame.font.SysFont("comicsans", 50)


class Nodo:
    def __init__(self, fila, col, ancho, total_filas, etiqueta):  # Corregido el constructor
        self.fila = fila
        self.col = col
        self.x = fila * ancho
        self.y = col * ancho
        self.color = BLANCO
        self.ancho = ancho
        self.total_filas = total_filas
        self.vecinos = []
        self.g = float("inf")
        self.h = 0
        self.f = float("inf")
        self.padre = None
        self.etiqueta = etiqueta

    def get_pos(self):
        return self.fila, self.col

    def es_pared(self):
        return self.color == NEGRO

    def es_inicio(self):
        return self.color == NARANJA

    def es_fin(self):
        return self.color == PURPURA

    def restablecer(self):
        self.color = BLANCO

    def hacer_inicio(self):
        self.color = NARANJA

    def hacer_pared(self):
        self.color = NEGRO

    def hacer_fin(self):
        self.color = PURPURA

    def dibujar(self, ventana):
        pygame.draw.rect(ventana, self.color, (self.x, self.y, self.ancho, self.ancho))

    def actualizar_vecinos(self, grid):
        self.vecinos = []
        # Movimiento abajo
        if self.fila < self.total_filas - 1 and not grid[self.fila + 1][self.col].es_pared():
            self.vecinos.append(grid[self.fila + 1][self.col])
        # Movimiento arriba
        if self.fila > 0 and not grid[self.fila - 1][self.col].es_pared():
            self.vecinos.append(grid[self.fila - 1][self.col])
        # Movimiento derecha
        if self.col < self.total_filas - 1 and not grid[self.fila][self.col + 1].es_pared():
            self.vecinos.append(grid[self.fila][self.col + 1])
        # Movimiento izquierda
        if self.col > 0 and not grid[self.fila][self.col - 1].es_pared():
            self.vecinos.append(grid[self.fila][self.col - 1])
        # Movimiento diagonal abajo-derecha
        if self.fila < self.total_filas - 1 and self.col < self.total_filas - 1 and not grid[self.fila + 1][self.col + 1].es_pared():
            self.vecinos.append(grid[self.fila + 1][self.col + 1])
        # Movimiento diagonal abajo-izquierda
        if self.fila < self.total_filas - 1 and self.col > 0 and not grid[self.fila + 1][self.col - 1].es_pared():
            self.vecinos.append(grid[self.fila + 1][self.col - 1])
        # Movimiento diagonal arriba-derecha
        if self.fila > 0 and self.col < self.total_filas - 1 and not grid[self.fila - 1][self.col + 1].es_pared():
            self.vecinos.append(grid[self.fila - 1][self.col + 1])
        # Movimiento diagonal arriba-izquierda
        if self.fila > 0 and self.col > 0 and not grid[self.fila - 1][self.col - 1].es_pared():
            self.vecinos.append(grid[self.fila - 1][self.col - 1])

    def __lt__(self, other):  # Corregido la comparación
        return self.f < other.f


def heuristica(nodo1, nodo2):
    x1, y1 = nodo1.get_pos()
    x2, y2 = nodo2.get_pos()
    return abs(x1 - x2) + abs(y1 - y2)


def reconstruir_camino(came_from, actual, dibujar):
    while actual in came_from:
        actual = came_from[actual]
        actual.color = VERDE
        dibujar()
        pygame.time.delay(50)


class ListaVentana:
    def __init__(self):  # Corregido el constructor
        self.root = tk.Tk()
        self.root.title("Listas Abierta y Cerrada")
        self.root.geometry("400x600")

        self.label_abierta = tk.Label(self.root, text="Lista Abierta:", font=("Arial", 14, "bold"))
        self.label_abierta.pack()
        self.text_abierta = tk.Text(self.root, height=15, width=50)
        self.text_abierta.pack()

        self.label_cerrada = tk.Label(self.root, text="Lista Cerrada:", font=("Arial", 14, "bold"))
        self.label_cerrada.pack()
        self.text_cerrada = tk.Text(self.root, height=15, width=50)
        self.text_cerrada.pack()

    def actualizar_abierta(self, abierta):
        self.text_abierta.delete(1.0, tk.END)
        for nodo in abierta:
            if nodo.padre:
                self.text_abierta.insert(tk.END, f"{nodo.padre.etiqueta} -> {nodo.etiqueta}\n")
            else:
                self.text_abierta.insert(tk.END, f"{nodo.etiqueta}\n")
        self.text_abierta.update_idletasks()

    def actualizar_cerrada(self, cerrada):
        self.text_cerrada.delete(1.0, tk.END)
        for nodo in cerrada:
            if nodo.padre:
                self.text_cerrada.insert(tk.END, f"{nodo.padre.etiqueta} -> {nodo.etiqueta}\n")
            else:
                self.text_cerrada.insert(tk.END, f"{nodo.etiqueta}\n")
        self.text_cerrada.update_idletasks()

    def iniciar(self):
        self.root.mainloop()


def a_star(dibujar, grid, inicio, fin, lista_ventana):
    cont = 0
    open_set = PriorityQueue()
    open_set.put((0, cont, inicio))
    came_from = {}

    inicio.g = 0
    inicio.f = heuristica(inicio, fin)

    open_set_hash = {inicio}
    closed_set = set()

    while not open_set.empty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

        actual = open_set.get()[2]
        open_set_hash.remove(actual)
        closed_set.add(actual)

        if actual == fin:
            reconstruir_camino(came_from, fin, dibujar)
            fin.hacer_fin()
            return True

        for vecino in actual.vecinos:
            diagonal = abs(vecino.fila - actual.fila) == 1 and abs(vecino.col - actual.col) == 1
            temp_g_score = actual.g + (1.4 if diagonal else 1)

            if temp_g_score < vecino.g:
                vecino.padre = actual
                came_from[vecino] = actual
                vecino.g = temp_g_score
                vecino.h = heuristica(vecino, fin)
                vecino.f = vecino.g + vecino.h
                if vecino not in open_set_hash and vecino not in closed_set:
                    cont += 1
                    open_set.put((vecino.f, cont, vecino))
                    open_set_hash.add(vecino)
                    vecino.color = ROJO

        dibujar()
        actual.color = AZUL
        pygame.time.delay(30)

        lista_ventana.actualizar_abierta(open_set_hash)
        lista_ventana.actualizar_cerrada(closed_set)

    return False


def crear_grid(filas, ancho):
    grid = []
    ancho_nodo = ancho // filas

    etiquetas = list(string.ascii_uppercase) + [str(i) for i in range(1, 1000)]
    etiqueta_idx = 0

    for i in range(filas):
        grid.append([])
        for j in range(filas):
            nodo = Nodo(i, j, ancho_nodo, filas, etiquetas[etiqueta_idx])
            etiqueta_idx += 1
            grid[i].append(nodo)
    return grid


def dibujar_grid(ventana, filas, ancho):
    ancho_nodo = ancho // filas
    for i in range(filas):
        pygame.draw.line(ventana, GRIS, (0, i * ancho_nodo), (ancho, i * ancho_nodo))
        for j in range(filas):
            pygame.draw.line(ventana, GRIS, (j * ancho_nodo, 0), (j * ancho_nodo, ancho))


def dibujar(ventana, grid, filas, ancho):
    ventana.fill(BLANCO)
    for fila in grid:
        for nodo in fila:
            nodo.dibujar(ventana)

    dibujar_grid(ventana, filas, ancho)
    pygame.display.update()


def obtener_click_pos(pos, filas, ancho):
    ancho_nodo = ancho // filas
    y, x = pos
    fila = y // ancho_nodo
    col = x // ancho_nodo
    return fila, col


def ejecutar_pygame(lista_ventana):
    ventana = pygame.display.set_mode((ANCHO_VENTANA, ANCHO_VENTANA))
    filas = 10  # Corregido la asignación de filas
    grid = crear_grid(filas, ANCHO_VENTANA)

    inicio = None
    fin = None
    corriendo = True

    while corriendo:
        dibujar(ventana, grid, filas, ANCHO_VENTANA)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                corriendo = False

            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                fila, col = obtener_click_pos(pos, filas, ANCHO_VENTANA)
                nodo = grid[fila][col]
                if not inicio and nodo != fin:
                    inicio = nodo
                    inicio.hacer_inicio()
                elif not fin and nodo != inicio:
                    fin = nodo
                    fin.hacer_fin()
                elif nodo != fin and nodo != inicio:
                    nodo.hacer_pared()

            elif pygame.mouse.get_pressed()[2]:
                pos = pygame.mouse.get_pos()
                fila, col = obtener_click_pos(pos, filas, ANCHO_VENTANA)
                nodo = grid[fila][col]
                nodo.restablecer()
                if nodo == inicio:
                    inicio = None
                elif nodo == fin:
                    fin = None

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and inicio and fin:
                    for fila in grid:
                        for nodo in fila:
                            nodo.actualizar_vecinos(grid)

                    a_star(lambda: dibujar(ventana, grid, filas, ANCHO_VENTANA), grid, inicio, fin, lista_ventana)

                if event.key == pygame.K_r:
                    grid = crear_grid(filas, ANCHO_VENTANA)
                    inicio = None
                    fin = None

                if event.key == pygame.K_q:
                    corriendo = False

    pygame.quit()


if __name__ == "__main__":  # Corregido la ejecución principal
    lista_ventana = ListaVentana()
    pygame_thread = Thread(target=ejecutar_pygame, args=(lista_ventana,))
    pygame_thread.start()
    lista_ventana.iniciar()

In [4]:
import pygame
from queue import PriorityQueue
import tkinter as tk
from threading import Thread
import string

# Configuraciones iniciales CODIGO EN VERIFICCIIJNOPI
ANCHO_VENTANA = 600

# Colores (RGB)
BLANCO = (255, 255, 255)
NEGRO = (0, 0, 0)
GRIS = (128, 128, 128)
VERDE = (0, 255, 0)
ROJO = (255, 0, 0)
NARANJA = (255, 165, 0)
PURPURA = (128, 0, 128)
AZUL = (0, 0, 255)

pygame.font.init()
FUENTE = pygame.font.SysFont("comicsans", 50)


class Nodo:
    def __init__(self, fila, col, ancho, total_filas, etiqueta):  # Corregido el constructor
        self.fila = fila
        self.col = col
        self.x = fila * ancho
        self.y = col * ancho
        self.color = BLANCO
        self.ancho = ancho
        self.total_filas = total_filas
        self.vecinos = []
        self.g = float("inf")
        self.h = 0
        self.f = float("inf")
        self.padre = None
        self.etiqueta = etiqueta

    def get_pos(self):
        return self.fila, self.col

    def es_pared(self):
        return self.color == NEGRO

    def es_inicio(self):
        return self.color == NARANJA

    def es_fin(self):
        return self.color == PURPURA

    def restablecer(self):
        self.color = BLANCO

    def hacer_inicio(self):
        self.color = NARANJA

    def hacer_pared(self):
        self.color = NEGRO

    def hacer_fin(self):
        self.color = PURPURA

    def dibujar(self, ventana):
        pygame.draw.rect(ventana, self.color, (self.x, self.y, self.ancho, self.ancho))

    def actualizar_vecinos(self, grid):
        self.vecinos = []
        # Movimiento abajo
        if self.fila < self.total_filas - 1 and not grid[self.fila + 1][self.col].es_pared():
            self.vecinos.append(grid[self.fila + 1][self.col])
        # Movimiento arriba
        if self.fila > 0 and not grid[self.fila - 1][self.col].es_pared():
            self.vecinos.append(grid[self.fila - 1][self.col])
        # Movimiento derecha
        if self.col < self.total_filas - 1 and not grid[self.fila][self.col + 1].es_pared():
            self.vecinos.append(grid[self.fila][self.col + 1])
        # Movimiento izquierda
        if self.col > 0 and not grid[self.fila][self.col - 1].es_pared():
            self.vecinos.append(grid[self.fila][self.col - 1])
        # Movimiento diagonal abajo-derecha
        if self.fila < self.total_filas - 1 and self.col < self.total_filas - 1 and not grid[self.fila + 1][self.col + 1].es_pared():
            self.vecinos.append(grid[self.fila + 1][self.col + 1])
        # Movimiento diagonal abajo-izquierda
        if self.fila < self.total_filas - 1 and self.col > 0 and not grid[self.fila + 1][self.col - 1].es_pared():
            self.vecinos.append(grid[self.fila + 1][self.col - 1])
        # Movimiento diagonal arriba-derecha
        if self.fila > 0 and self.col < self.total_filas - 1 and not grid[self.fila - 1][self.col + 1].es_pared():
            self.vecinos.append(grid[self.fila - 1][self.col + 1])
        # Movimiento diagonal arriba-izquierda
        if self.fila > 0 and self.col > 0 and not grid[self.fila - 1][self.col - 1].es_pared():
            self.vecinos.append(grid[self.fila - 1][self.col - 1])

    def __lt__(self, other):  # Corregido la comparación
        return self.f < other.f


def heuristica(nodo1, nodo2):
    x1, y1 = nodo1.get_pos()
    x2, y2 = nodo2.get_pos()
    return abs(x1 - x2) + abs(y1 - y2)


def reconstruir_camino(came_from, actual, dibujar):
    while actual in came_from:
        actual = came_from[actual]
        actual.color = VERDE
        dibujar()
        pygame.time.delay(50)


class ListaVentana:
    def __init__(self):  # Corregido el constructor
        self.root = tk.Tk()
        self.root.title("Listas Abierta y Cerrada")
        self.root.geometry("400x600")

        self.label_abierta = tk.Label(self.root, text="Lista Abierta:", font=("Arial", 14, "bold"))
        self.label_abierta.pack()
        self.text_abierta = tk.Text(self.root, height=15, width=50)
        self.text_abierta.pack()

        self.label_cerrada = tk.Label(self.root, text="Lista Cerrada:", font=("Arial", 14, "bold"))
        self.label_cerrada.pack()
        self.text_cerrada = tk.Text(self.root, height=15, width=50)
        self.text_cerrada.pack()

    def actualizar_abierta(self, abierta):
        self.text_abierta.delete(1.0, tk.END)
        for nodo in abierta:
            if nodo.padre:
                self.text_abierta.insert(tk.END, f"{nodo.padre.etiqueta} -> {nodo.etiqueta}\n")
            else:
                self.text_abierta.insert(tk.END, f"{nodo.etiqueta}\n")
        self.text_abierta.update_idletasks()

    def actualizar_cerrada(self, cerrada):
        self.text_cerrada.delete(1.0, tk.END)
        for nodo in cerrada:
            if nodo.padre:
                self.text_cerrada.insert(tk.END, f"{nodo.padre.etiqueta} -> {nodo.etiqueta}\n")
            else:
                self.text_cerrada.insert(tk.END, f"{nodo.etiqueta}\n")
        self.text_cerrada.update_idletasks()

    def iniciar(self):
        self.root.mainloop()


def a_star(dibujar, grid, inicio, fin, lista_ventana):
    cont = 0
    open_set = PriorityQueue()
    open_set.put((0, cont, inicio))
    came_from = {}

    inicio.g = 0
    inicio.f = heuristica(inicio, fin)

    open_set_hash = {inicio}
    closed_set = set()

    while not open_set.empty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

        actual = open_set.get()[2]
        open_set_hash.remove(actual)
        closed_set.add(actual)

        if actual == fin:
            reconstruir_camino(came_from, fin, dibujar)
            fin.hacer_fin()
            return True

        for vecino in actual.vecinos:
            diagonal = abs(vecino.fila - actual.fila) == 1 and abs(vecino.col - actual.col) == 1
            temp_g_score = actual.g + (1.4 if diagonal else 1)

            if temp_g_score < vecino.g:
                vecino.padre = actual
                came_from[vecino] = actual
                vecino.g = temp_g_score
                vecino.h = heuristica(vecino, fin)
                vecino.f = vecino.g + vecino.h
                if vecino not in open_set_hash and vecino not in closed_set:
                    cont += 1
                    open_set.put((vecino.f, cont, vecino))
                    open_set_hash.add(vecino)
                    vecino.color = ROJO

        dibujar()
        actual.color = AZUL
        pygame.time.delay(30)

        lista_ventana.actualizar_abierta(open_set_hash)
        lista_ventana.actualizar_cerrada(closed_set)

    return False


def crear_grid(filas, ancho):
    grid = []
    ancho_nodo = ancho // filas

    etiquetas = list(string.ascii_uppercase) + [str(i) for i in range(1, 1000)]
    etiqueta_idx = 0

    for i in range(filas):
        grid.append([])
        for j in range(filas):
            nodo = Nodo(i, j, ancho_nodo, filas, etiquetas[etiqueta_idx])
            etiqueta_idx += 1
            grid[i].append(nodo)
    return grid


def dibujar_grid(ventana, filas, ancho):
    ancho_nodo = ancho // filas
    for i in range(filas):
        pygame.draw.line(ventana, GRIS, (0, i * ancho_nodo), (ancho, i * ancho_nodo))
        for j in range(filas):
            pygame.draw.line(ventana, GRIS, (j * ancho_nodo, 0), (j * ancho_nodo, ancho))


def dibujar(ventana, grid, filas, ancho):
    ventana.fill(BLANCO)
    for fila in grid:
        for nodo in fila:
            nodo.dibujar(ventana)

    dibujar_grid(ventana, filas, ancho)
    pygame.display.update()


def obtener_click_pos(pos, filas, ancho):
    ancho_nodo = ancho // filas
    y, x = pos
    fila = y // ancho_nodo
    col = x // ancho_nodo
    return fila, col


def ejecutar_pygame(lista_ventana):
    ventana = pygame.display.set_mode((ANCHO_VENTANA, ANCHO_VENTANA))
    filas = 10  # Corregido la asignación de filas
    grid = crear_grid(filas, ANCHO_VENTANA)

    inicio = None
    fin = None
    corriendo = True

    while corriendo:
        dibujar(ventana, grid, filas, ANCHO_VENTANA)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                corriendo = False

            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                fila, col = obtener_click_pos(pos, filas, ANCHO_VENTANA)
                nodo = grid[fila][col]
                if not inicio and nodo != fin:
                    inicio = nodo
                    inicio.hacer_inicio()
                elif not fin and nodo != inicio:
                    fin = nodo
                    fin.hacer_fin()
                elif nodo != fin and nodo != inicio:
                    nodo.hacer_pared()

            elif pygame.mouse.get_pressed()[2]:
                pos = pygame.mouse.get_pos()
                fila, col = obtener_click_pos(pos, filas, ANCHO_VENTANA)
                nodo = grid[fila][col]
                nodo.restablecer()
                if nodo == inicio:
                    inicio = None
                elif nodo == fin:
                    fin = None

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and inicio and fin:
                    for fila in grid:
                        for nodo in fila:
                            nodo.actualizar_vecinos(grid)

                    a_star(lambda: dibujar(ventana, grid, filas, ANCHO_VENTANA), grid, inicio, fin, lista_ventana)

                if event.key == pygame.K_r:
                    grid = crear_grid(filas, ANCHO_VENTANA)
                    inicio = None
                    fin = None

                if event.key == pygame.K_q:
                    corriendo = False

    pygame.quit()


if __name__ == "__main__":  # Corregido la ejecución principal
    lista_ventana = ListaVentana()
    pygame_thread = Thread(target=ejecutar_pygame, args=(lista_ventana,))
    pygame_thread.start()
    lista_ventana.iniciar()